In [1]:
from azureml.core import Workspace, Experiment

In [2]:
#ws = Workspace.get(name="udacity-project")
ws = Workspace.from_config()

In [3]:
exp = Experiment(workspace=ws, name="udacity-project-experiment")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: azmldatad01
Azure region: eastus2
Subscription id: 8723c91c-b9c0-4b54-b0e0-8561d583e19b
Resource group: rsgdatad01


In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

cluster_name = "cpu-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config) ## creating the cluster

compute_target.wait_for_completion(show_output=True)

Creating a new compute target...
Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [5]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os
import shutil

# Specify parameter sampler
ps = RandomParameterSampling({
    "--C": choice(0.01, 0.1, 0.5, 1.0, 1.5, 2.0, 3.0),
    "--max_iter": choice(50, 100, 150, 200)
}
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

script_path = "./training"
script_name = 'train.py'

if "training" not in os.listdir():
    os.mkdir(script_path)
    shutil.copy(script_name, script_path)

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory=script_path, entry_script=script_name, compute_target=compute_target)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                                    hyperparameter_sampling=ps,
                                    policy=policy,
                                    primary_metric_name="Accuracy",
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=24,
                                    max_concurrent_runs=4
                                    )

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [6]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(config=hyperdrive_config, show_output=True)

In [7]:
hyperdrive_run

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project-experiment,HD_f6d5b016-d770-47cf-bfa4-61d1e59014ca,hyperdrive,Running,Link to Azure Machine Learning studio,Link to Documentation


In [19]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
best_run_details = best_run.get_details()

print("Best Run ID: {0}".format(best_run.id))
print("Accuracy: {0:.3f}".format(best_run_metrics["Accuracy"]))
print("Parameters: {0}".format(best_run_details["runDefinition"]["arguments"]))

Best Run ID: HD_f6d5b016-d770-47cf-bfa4-61d1e59014ca_17
Accuracy: 0.909
Parameters: ['--C', '2', '--max_iter', '50']


In [27]:
joblib.dump(value=best_run.id, filename="./outputs/best_model.joblib")
best_run.upload_file("outputs/best_model.joblib", "outputs/best_model.joblib")
best_run.register_model("best_model", model_path="outputs/best_model.joblib")

In [29]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds_url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=ds_url)
df = ds.to_pandas_dataframe()
df.head(10)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,57,technician,married,high.school,no,no,yes,cellular,may,mon,...,1,999,1,failure,-1.8,92.893,-46.2,1.299,5099.1,no
1,55,unknown,married,unknown,unknown,yes,no,telephone,may,thu,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,no
2,33,blue-collar,married,basic.9y,no,no,no,cellular,may,fri,...,1,999,1,failure,-1.8,92.893,-46.2,1.313,5099.1,no
3,36,admin.,married,high.school,no,no,no,telephone,jun,fri,...,4,999,0,nonexistent,1.4,94.465,-41.8,4.967,5228.1,no
4,27,housemaid,married,high.school,no,yes,no,cellular,jul,fri,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,no
5,58,retired,married,professional.course,no,yes,yes,cellular,jul,fri,...,1,999,0,nonexistent,1.4,93.918,-42.7,4.962,5228.1,no
6,48,services,married,high.school,unknown,yes,no,telephone,may,wed,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.856,5191.0,no
7,51,admin.,divorced,university.degree,unknown,yes,no,cellular,aug,thu,...,7,999,0,nonexistent,1.4,93.444,-36.1,4.962,5228.1,no
8,24,entrepreneur,married,university.degree,no,yes,yes,telephone,jun,wed,...,4,999,0,nonexistent,1.4,94.465,-41.8,4.962,5228.1,no
9,36,technician,divorced,professional.course,no,yes,yes,cellular,jul,mon,...,4,999,0,nonexistent,1.4,93.918,-42.7,4.962,5228.1,no


In [30]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [31]:
import pandas as pd
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=66)

df_train = pd.concat([X_train, y_train], axis=1).reset_index(drop=True)
df_train.to_csv("bank_train_data.csv", index=False)
df_train.head()

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y
0,50,0,0,0,0,6,1,408,2,999,...,1,0,0,0,0,0,1,0,0,0
1,31,0,0,1,0,3,1,145,1,999,...,0,0,0,0,0,0,0,1,0,0
2,43,1,0,0,0,5,4,134,1,999,...,1,1,0,0,0,0,0,0,0,0
3,34,0,0,1,1,7,4,470,4,999,...,0,0,0,0,0,0,0,0,1,0
4,25,1,0,0,0,5,1,480,1,999,...,1,1,0,0,0,0,0,0,0,0


In [32]:
if not os.path.isdir("dataset"):
    os.mkdir("./dataset")
    shutil.copy("bank_train_data.csv", "./dataset")
    
data_store = ws.get_default_datastore()
data_store.upload(src_dir="./dataset", target_path="bankmarketing", overwrite=True, show_progress=True)

# Uploading the training data as a tabular dataset
ds_train = TabularDatasetFactory.from_delimited_files(path=data_store.path("bankmarketing/bank_train_data.csv"))


Uploading an estimated of 1 files
Uploading ./dataset/bank_train_data.csv
Uploaded ./dataset/bank_train_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


In [33]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    compute_target=compute_target,
    training_data=ds_train,
    label_column_name="y",
    n_cross_validations=5)

In [34]:
# Submit your automl run

exp_2 = Experiment(ws, "automl-experiment")
run_2 = exp_2.submit(automl_config, show_output=False)

Running on remote.


In [35]:
exp_2

Name,Workspace,Report Page,Docs Page
automl-experiment,azmldatad01,Link to Azure Machine Learning studio,Link to Documentation


In [36]:
run_2

Experiment,Id,Type,Status,Details Page,Docs Page
automl-experiment,AutoML_3089d355-7ff5-4a53-853d-868d633c4320,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


In [47]:
# Retrieve and save your best automl model.

best_run, model = run_2.get_output()
joblib.dump(model, "outputs/best_automl_model.joblib")
best_run.upload_file("outputs/best_automl_model.joblib", "outputs/best_automl_model.joblib")
best_run.register_model("auto-ml-model" , model_path = "outputs/best_automl_model.joblib")


Model(workspace=Workspace.create(name='azmldatad01', subscription_id='8723c91c-b9c0-4b54-b0e0-8561d583e19b', resource_group='rsgdatad01'), name=auto-ml-model, id=auto-ml-model:2, version=2, tags={}, properties={})

---